In [2]:
import pickle
import os 
import glob
import random
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import plyfile
import numpy as np
import open3d as o3d
import h5py
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import open3d as o3d
from sklearn.decomposition import PCA
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances

import cv2
import numpy as np

import os.path as osp
import sys
ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import evaluation, scan3r


data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
#scan_id= "38770c95-86d7-27b8-8717-3485b411ddc7" #is reference scan  since it is a reference scan everything shouls be correctly hit
curr_scan_id =  "02b33dfb-be2b-2d54-92d2-cd012b2b3c40"#"02b33e01-be2b-2d54-93fb-4145a709cec5" 
new_scan_id =  "fcf66d8a-622d-291c-8429-0e1109c6bb26" #"fcf66d8a-622d-291c-8429-0e1109c6bb26"



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
/local/home/ekoller/BT


In [14]:
#for a given scene get the colours of the differnt object_ids
def get_id_colours(data_dir,scan_id):
    #access the mesh file to get the colour of the ids
    mesh_file = osp.join(data_dir,"scenes", scan_id, "labels.instances.annotated.v2.ply")
    ply_data = plyfile.PlyData.read(mesh_file)
    # Extract vertex data
    vertices = ply_data['vertex']
    vertex_count = len(vertices)
    
    # Initialize dictionary to store object_id -> color mappings
    object_colors = {}
    
   # Iterate through vertices
    for i in range(vertex_count):
        vertex = vertices[i]
        object_id = vertex['objectId']
        color = (vertex['red'], vertex['green'], vertex['blue'])
        
        # Check if object_id already in dictionary, otherwise initialize a Counter
        if object_id in object_colors:
            object_colors[object_id][color] += 1
        else:
            object_colors[object_id] = Counter({color: 1})
    
    # Convert Counter to dictionary with most frequent color
    for object_id, color_counter in object_colors.items():
        most_common_color = color_counter.most_common(1)[0][0]
        object_colors[object_id] = np.array(most_common_color)
    
    return object_colors

# colors = get_id_colours(data_dir,curr_scan_id)
# print("cur scan id colors", colors)
# colors = get_id_colours(data_dir,new_scan_id)
# print("new scan id colors", colors.keys())

cur scan id colors {3: array([255, 187, 120], dtype=uint8), 10: array([148, 103, 189], dtype=uint8), 17: array([158, 218, 229], dtype=uint8), 6: array([ 44, 160,  44], dtype=uint8), 4: array([255, 127,  14], dtype=uint8), 16: array([188, 189,  34], dtype=uint8), 2: array([ 31, 119, 180], dtype=uint8), 7: array([255, 152, 150], dtype=uint8), 15: array([219, 219, 141], dtype=uint8), 14: array([227, 119, 194], dtype=uint8), 1: array([174, 199, 232], dtype=uint8), 11: array([196, 156, 148], dtype=uint8), 0: array([0, 0, 0], dtype=uint8), 22: array([202, 185,  52], dtype=uint8), 8: array([214,  39,  40], dtype=uint8), 9: array([197, 176, 213], dtype=uint8), 30: array([140, 153, 101], dtype=uint8), 28: array([ 91, 163, 138], dtype=uint8), 12: array([140,  86,  75], dtype=uint8), 26: array([ 78,  71, 183], dtype=uint8), 24: array([200,  54, 131], dtype=uint8), 25: array([ 92, 193,  61], dtype=uint8), 5: array([152, 223, 138], dtype=uint8), 29: array([153,  98, 156], dtype=uint8), 21: array([1

In [4]:
def acces_predicted_objects(scan_id):
    all_centers = {}
    filename = osp.join("/media/ekoller/T7/Predicted_Centers", scan_id + ".pkl")
    with h5py.File(filename, 'r') as h5file:
        for obj_id in h5file.keys():
            obj_group = h5file[obj_id]
            center = np.array(obj_group['center'])
            points = np.array(obj_group['points'])
            votes = int(obj_group['votes'][()])
            
            # Add it back to the dictionary
            all_centers[int(obj_id)] = {
                'center': center,
                'points': points,
                'votes': votes
            }
    return all_centers

acces_predicted_objects(new_scan_id)

{0: {'center': array([[-0.4760977 ,  1.35325495,  0.0927426 ]]),
  'points': array([[-0.43350216,  1.47394601,  0.37751036],
         [-0.40853397,  1.17425119, -0.00435107],
         [-0.37732428,  1.48571919,  0.3812295 ],
         ...,
         [-0.58517792,  1.14374966,  0.21411133],
         [-1.35839904,  2.17283871, -0.37604445],
         [-0.5894311 ,  2.18545082, -0.36533461]]),
  'votes': 16}}

this segment displays the predicted pointcloud and the ground truth point cloud

In [27]:
# create the newly predicted pointcloud
all_centers = acces_predicted_objects( new_scan_id)
#initialize the final pointcloud

final_point_cloud = o3d.geometry.PointCloud()

#colour
obj_colors = get_id_colours(data_dir, new_scan_id)
print("colors dict", obj_colors)

# Iterate over all objects in all_centers
for obj_id, data in all_centers.items():
    print("hey object added" , obj_id)
    points = data['points']  # Get points
    color = np.array(obj_colors[obj_id], dtype=np.float32)
    color = np.reshape(color, (3,))
    color = color / 255.0

    #color = np.array(obj_colors[obj_id])/255.0 # Generate a unique color for this object ID
    #color = np.random.rand(3)
    # Create a point cloud for the current object's points
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    
    # Create a color array of the same size as the number of points, and assign the same color to all points
    #colors = np.tile(color, (points.shape[0], 1))  # Shape (num_points, 3)
    point_cloud.colors = o3d.utility.Vector3dVector(np.tile(color, (len(points), 1)))

    #colors = np.asarray(point_cloud.colors)
    
    # # Scale the colors by the brightness factor
    # adjusted_colors = np.clip(colors * 1.3, 0, 1)  # Clip to ensure values are in the [0, 1] range
    
    # Update the point cloud's colors
    # point_cloud.colors = o3d.utility.Vector3dVector(adjusted_colors)

    # Add the current object's point cloud to the final point cloud
    final_point_cloud += point_cloud



#also access the ground truth pointclouds
pklfile = osp.join('/local/home/ekoller/R3Scan', 'files', 'orig', 'data', '{}.pkl'.format(new_scan_id))

with open(pklfile, "rb") as f:
    # Load the data from the pickle file
    data = pickle.load(f)
    
    # Extract object points and IDs from the pickle data
    pointclouds = data['obj_points'][512]
    object_ids = data['objects_id']
    
    # Create a new point cloud object for the .pkl data
    pkl_point_cloud = o3d.geometry.PointCloud()

    # Iterate over all the points and their object IDs
    for obj_id, points in zip(object_ids, pointclouds):
        # Assign a default color (e.g., black) to these objects
        print("object id", obj_id)
        color = obj_colors[obj_id]/225.0
        print("clour", obj_colors[obj_id]/225.0)
        # Create a point cloud for this object's points
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        pcd.colors = o3d.utility.Vector3dVector(np.tile(color, (points.shape[0], 1)))  # Assign the color

        # Add this point cloud to the combined pkl_point_cloud
        pkl_point_cloud += pcd

colors = np.asarray(pkl_point_cloud.colors)

# Scale the colors by the brightness factor
adjusted_colors = np.clip(colors *0.7, 0, 1)  # Clip to ensure values are in the [0, 1] range

# Update the point cloud's colors
pkl_point_cloud.colors = o3d.utility.Vector3dVector(adjusted_colors)
# Now combine both point clouds (final_point_cloud and pkl_point_cloud) for visualization
combined_point_cloud = final_point_cloud + pkl_point_cloud

# Visualize both together using Open3D
o3d.visualization.draw_geometries([pkl_point_cloud + final_point_cloud], window_name="Combined Point Clouds from Scan and PKL")



#o3d.visualization.draw_geometries([final_point_cloud], window_name="Combined Point Cloud")



colors dict {13: array([247, 182, 210], dtype=uint8), 20: array([ 66, 188, 102], dtype=uint8), 1: array([174, 199, 232], dtype=uint8), 4: array([255, 127,  14], dtype=uint8), 100: array([191,  63,  78], dtype=uint8), 6: array([ 44, 160,  44], dtype=uint8), 14: array([227, 119, 194], dtype=uint8), 15: array([219, 219, 141], dtype=uint8), 18: array([ 23, 190, 207], dtype=uint8), 29: array([153,  98, 156], dtype=uint8), 5: array([152, 223, 138], dtype=uint8), 8: array([214,  39,  40], dtype=uint8), 16: array([188, 189,  34], dtype=uint8), 105: array([153, 101, 134], dtype=uint8), 0: array([0, 0, 0], dtype=uint8)}
hey object added 0
object id 1
clour [0.77333333 0.88444444 1.03111111]
object id 4
clour [1.13333333 0.56444444 0.06222222]
object id 5
clour [0.67555556 0.99111111 0.61333333]
object id 6
clour [0.19555556 0.71111111 0.19555556]
object id 8
clour [0.95111111 0.17333333 0.17777778]
object id 105
clour [0.68       0.44888889 0.59555556]
object id 13
clour [1.09777778 0.80888889 0